# convnet 各層のフィルタ可視化、特徴強度画像作成

In [2]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

input_file = "kawasemi.jpg"
outout_dir = "filter-intensities"
model = VGG16(weights="imagenet")

Using TensorFlow backend.


In [4]:
from matplotlib.cm import get_cmap
from keras.preprocessing import image

cm = get_cmap("inferno")

def heatmap(activations):
  """活性化強度 (0.0-1.0) の画像をヒートマップ化し 0-255 の RGB 画像として扱えるように変換"""
  return (cm(activations / np.max(activations)) * 255).astype(np.int8).transpose((0, 3, 1, 2, 4))[:, :, :, :, :3]

def concat_images(images, spacing=1, space_color=0):
  """リスト化されている画像を縦横に並べた 1 枚の画像を作成する"""
  n_images = len(images)
  rows = math.ceil(math.sqrt(n_images))
  cols = math.ceil(n_images / rows)
  vstack = []
  for y in range(cols):
    size = min(rows, n_images - y * cols)
    hstack = []
    for x in range(rows):
      if x is not 0:
        hstack.append(np.full((images[x-1].shape[0], spacing, 3), space_color, dtype=np.uint8))
      i = y * cols + x
      if i < len(images):
        hstack.append(images[i].astype(np.uint8))
      else:
        hstack.append(np.zeros(images[0].shape, dtype=np.uint8))
    row = cv2.hconcat(hstack)
    vstack.append(row)
    if y + 1 < cols:
      vstack.append(np.full((spacing, row.shape[1], 3), space_color, dtype=np.uint8))
  return Image.fromarray(cv2.vconcat(vstack), "RGB")

def deprocess_image(x):
  # テンソルを正規化: 中心 0、標準偏差 0.1 に調整
  x -= x.mean()
  x /= (x.std() + 1e-5)
  x *= 0.1

  # [0,1] 範囲にクリッピング
  x += 0.5
  x = np.clip(x, 0, 1)

  # RGB に変換
  x *= 255
  return np.clip(x, 0, 255).astype(np.uint8)

def generate_pattern(layer_name, filter_index, size=150):
  layer_output = model.get_layer(layer_name).output
  loss = K.mean(layer_output[:, :, :, filter_index])
  grads = K.gradients(loss, model.input)[0]
  grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
  iter = K.function([model.input], [loss, grads])
  input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.
  step = 1.
  for i in range(40):
    loss_value, grads_value = iter([input_img_data])
    input_img_data += grads_value * step
  img = input_img_data[0]
  return deprocess_image(img)

def visualize_activation(activations):
  imgs = heatmap(activations)
  return image.img_to_array(concat_images(imgs, space_color=0x20)) 
